In [1]:
from keras.applications import vgg16
from keras.models import Model,load_model
from keras.layers import Input, Conv2D, Conv2DTranspose,AveragePooling2D, MaxPooling2D,UpSampling2D,LeakyReLU,Softmax, concatenate, Dropout,BatchNormalization,Activation
from keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from warnings import filterwarnings
from tensorflow.keras.optimizers import Adam
import zipfile
import numpy as np
import  os
import glob
import PIL
import cv2
from tensorflow.python.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from matplotlib import pyplot as plt
from tensorflow.keras.layers.experimental.preprocessing import Resizing

C:\ProgramData\Anaconda3\envs\DL\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
class LoadData:
    def __init__(self):
        self.DataImage = []
        self.DataLabel = []
        self.DataID=[]
        self.LabelImage=[]
    #Tke image and label from Folder contain folder child (name label=name folder child)
    def CNNFromFolder(self,pathFolder,imageType,size):
        for dirPath in glob.glob(pathFolder+"/*"):
            label = dirPath.split("/")[-1]
            for image_path in glob.glob(os.path.join(dirPath,"*."+imageType)):
                image = cv2.imread(image_path,cv2.IMREAD_COLOR)
                image = cv2.resize(image, (size[0], size[1]),interpolation = cv2.INTER_NEAREST)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                self.DataImage.append(image)
                self.DataLabel.append(label)
        self.DataImage=np.array(self.DataImage)
        self.DataLabel=np.array(self.DataLabel)
        LabelToID = {v:k for k,v in enumerate(np.unique(self.DataLabel)) }
        IDToLabel = {v:k for k,v in LabelToID.items() }
        self.DataID=np.array([LabelToID[i] for i in self.DataLabel])
    #take image x and image y from Folder image y have name diff by diffStringLabel
    def SegmentationFromFolder(self,pathFolder,imageType_X,imageType_Y,size,diffStringLabel):
         for image_path_x in glob.glob(os.path.join(pathFolder,"*."+imageType_X)):
                if(~image_path_x.__contains__(diffStringLabel)):
                    image_x=cv2.imread(image_path_x,cv2.IMREAD_COLOR)
                    for image_path_y in glob.glob(os.path.join(pathFolder,"*."+imageType_Y)):
                        image_path_not_ext=os.path.splitext(image_path_x)[0]
                        if(image_path_y==image_path_not_ext+diffStringLabel+"."+imageType_Y):
                            image_y=cv2.imread(image_path_y,cv2.IMREAD_COLOR)
                            image_x = cv2.resize(image_x, (size[0], size[1]),interpolation = cv2.INTER_NEAREST)
                            image_x = cv2.cvtColor(image_x, cv2.COLOR_BGR2RGB)
                            image_y= cv2.resize(image_y, (size[0], size[1]),interpolation = cv2.INTER_NEAREST)
                            image_y = cv2.cvtColor(image_y, cv2.COLOR_BGR2RGB)
                            self.DataImage.append(image_x)
                            self.LabelImage.append(image_y)
                            image_x=None
                            image_y=None
                            break
         self.DataImage=np.array(self.DataImage)
         self.LabelImage=np.array(self.LabelImage)
         return np.array(self.DataImage),np.array(self.LabelImage)
      

In [ ]:
# Segmentation
class ProcessDataSegmatation:
    def __init__(self):
        self.DataX=[]
        self.DataY=[]
    def RandomRotation(self,x_image, y_image):
        rows_x,cols_x, chl_x = x_image.shape
        rows_y,cols_y,chl_y = y_image.shape
        rand_num = np.random.randint(-45,45)
        M1 = cv2.getRotationMatrix2D((cols_x/2,rows_x/2),rand_num,1)
        M2 = cv2.getRotationMatrix2D((cols_y/2,rows_y/2),rand_num,1)
        x_image = cv2.warpAffine(x_image,M1,(cols_x,rows_x))
        y_image = cv2.warpAffine(y_image.astype('float'),M2,(cols_y,rows_y))
        M1=None
        M2=None
        rand_num=None
        return x_image, y_image.astype('int')
    def Rotation(self,x_image, y_image,angle):
        rows_x,cols_x, chl_x = x_image.shape
        rows_y,cols_y,chl_y = y_image.shape
        rand_num = angle
        M1 = cv2.getRotationMatrix2D((cols_x/2,rows_x/2),rand_num,1)
        M2 = cv2.getRotationMatrix2D((cols_y/2,rows_y/2),rand_num,1)
        x_image = cv2.warpAffine(x_image,M1,(cols_x,rows_x))
        y_image = cv2.warpAffine(y_image.astype('float'),M2,(cols_y,rows_y))
        M1=None
        M2=None
        rand_num=None
        return x_image, y_image.astype('int')
    def HorizontalFlip(self,x_image, y_image):
        x_image = cv2.flip(x_image, 1)
        y_image = cv2.flip(y_image.astype('float'), 1)
        return x_image, y_image.astype('int')
    def ImgAugmentation(self,x_train, y_train):
        x_rotat= []
        y_rotat= []
        x_flip = []
        y_flip = []
        x_nois = []
        for idx in range(len(x_train)):
            x=None
            y=None
            x,y = self.RandomRotation(x_train[idx], y_train[idx])
            x_rotat.append(x)
            y_rotat.append(y)
            x=None
            y=None
            x,y = self.HorizontalFlip(x_train[idx], y_train[idx])
            x_flip.append(x)
            y_flip.append(y)
            x=None
            y=None
        return np.array(x_rotat), np.array(y_rotat), np.array(x_flip), np.array(y_flip)
    def DataAugmentation(self,x_train,y_train):
        x_rotated, y_rotated, x_flipped, y_flipped = self.ImgAugmentation(x_train, y_train)
        x_train_full = np.concatenate([x_train, x_rotated, x_flipped])
        y_train_full = np.concatenate([y_train, y_rotated, y_flipped])
        x_rotated=None
        y_rotated=None
        x_flipped=None
        y_flipped=None
        self.DataX=x_train_full
        self.DataY=y_train_full
        return x_train_full,y_train_full
    
class ProcessDataCNN:
    def __init__(self):
        self.DataX=[]
        self.DataY=[]
    def RandomRotation(x_image, y_label):
        rows_x,cols_x, chl_x = x_image.shape
        rand_num = np.random.randint(-45,45)
        M1 = cv2.getRotationMatrix2D((cols_x/2,rows_x/2),rand_num,1)
        M2 = cv2.getRotationMatrix2D((cols_y/2,rows_y/2),rand_num,1)
        x_image = cv2.warpAffine(x_image,M1,(cols_x,rows_x))
        return x_image, y_label
    def HorizontalFlip(x_image, y_label):
        x_image = cv2.flip(x_image, 1)
        return x_image, y_label
    def ImgAugmentation(x_train, y_train):
        x_rotat= []
        y_rotat= []
        x_flip = []
        y_flip = []
        x_nois = []
        for idx in range(len(x_train)):
            x,y = self.RandomRotation(x_train[idx], y_train[idx])
            x_rotat.append(x)
            y_rotat.append(y)
            x,y = self.HorizontalFlip(x_train[idx], y_train[idx])
            x_flip.append(x)
            y_flip.append(y)
        return np.array(x_rotat), np.array(y_rotat), np.array(x_flip), np.array(y_flip)
    def DataAugmentation(x_train,y_train):
        x_rotated, y_rotated, x_flipped, y_flipped = self.ImgAugmentation(x_train, y_train)
        x_train_full = np.concatenate([x_train, x_rotated, x_flipped])
        y_train_full = np.concatenate([y_train, y_rotated, y_flipped])
        self.DataX=x_train_full
        self.DataY=y_train_full
        return x_train_full,y_train_full


In [ ]:
class ImageLabel:
    def __init__(self,ListColor,ListID,ListName):
        self.ListColor=ListColor
        self.ListID=ListID
        try:
            self.code2id = {v:k for k,v in enumerate(ListColor)}
            self.id2code = {k:v for k,v in enumerate(ListColor)}
            self.name2id = {v:k for k,v in enumerate(ListName)}
            self.id2name = {k:v for k,v in enumerate(ListName)}
        except:
            print("Error")
    def ImageColorToImageID(self,Image):
        i=0
        im=np.zeros(Image.shape)
        for color in self.ListColor:
            ID=self.ListID[i]
            red, green, blue = Image[:,:,0], Image[:,:,1], Image[:,:,2] 
            mask = (red == color[0]) & (green == color[1]) & (blue == color[2])
            im[:,:,:3][mask] = [ID[0], ID[1], ID[2]]
            i=i+1
        return im;
    def ImageIDToColor(self,Image):
        i=0;
        im=np.zeros(Image.shape)
        for ID in self.ListID:
            color=self.ListColor[i]
            id0, id1, id2 = Image[:,:,0], Image[:,:,1], Image[:,:,2] 
            mask = (id0 == ID[0]) & (id1 == ID[1]) & (id2 == ID[2])
            im[:,:,:3][mask] = [color[0], color[1], color[2]]
            i=i+1
            mask=None
            id0=None
            id1=None
            id2=None
            color=None
        return im;
    def TranferListImageColorToListImageID(self,ListImage):
        ListImageID=[]
        for Image in ListImage:
            image=self.ImageColorToImageID(Image);
            ListImageID.append(image);
            image=None
        ListImageID=np.array(ListImageID)
        return ListImageID
    def TranferListImageIDToListImageColor(self,ListImage):
        ListImageColor=[]
        for Image in ListImage:
            image=self.ImageIDToImageColor(Image);
            ListImagecolor.append(image);
            image=None
        ListImageColor=np.array(ListImageColor)
        return ListImageColor
    def RGB_To_OneHot(self,rgb_image):
        colormap = self.id2code
        '''Function to one hot encode RGB mask labels
        Inputs: 
            rgb_image - image matrix (eg. 256 x 256 x 3 dimension numpy ndarray)
            colormap - dictionary of color to label id
        Output: One hot encoded image of dimensions (height x width x num_classes) where num_classes = len(colormap)
        '''
        num_classes = len(colormap)
        shape = rgb_image.shape[:2]+(num_classes,)
        encoded_image = np.zeros( shape, dtype=np.int8 )
        for i, cls in enumerate(colormap):
            encoded_image[:,:,i] = np.all(rgb_image.reshape( (-1,3) ) == colormap[i], axis=1).reshape(shape[:2])
        return encoded_image
    def OneHot_To_RGB(self,onehot ):
        colormap = self.id2code
        '''Function to decode encoded mask labels
            Inputs: 
            onehot - one hot encoded image matrix (height x width x num_classes)
            colormap - dictionary of color to label id
        Output: Decoded RGB image (height x width x 3) 
        '''
        single_layer = np.argmax(onehot, axis=-1)
        output = np.zeros( onehot.shape[:2]+(3,) )
        for k in colormap.keys():
            output[single_layer==k] = colormap[k]
        return np.uint8(output)
    def ListImageToOneHot(self,ListImageColor):
        ListImageID=[]
        for imgColor in ListImageColor:
            imgID=self.RGB_To_OneHot(imgColor)
            ListImageID.append(imgID)
        return np.array(ListImageID)
    def ListOneHotToImage(self,ListImageID):
        ListImageColor=[]
        for imgID in ListImageID:
            imgColor=self.OneHot_To_RGB(imgID)
            ListImageColor.append(imgColor)
        return np.array(ListImageColor)
            
    
    

In [ ]:
class PreProcessImageWithTensorflow:
    def __init__(self):
        self.train_datagen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True,
            rotation_range=360,
            width_shift_range=0.2,
            height_shift_range=0.2,)
        self.test_datagen = ImageDataGenerator(rescale=1./255)
        
        self.seg_trainX_datagen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True,
            rotation_range=360,
            width_shift_range=0.2,
            height_shift_range=0.2,)
        self.seg_trainY_datagen = ImageDataGenerator(
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True,
            rotation_range=360,
            width_shift_range=0.2,
            height_shift_range=0.2,)
        self.seg_testX_datagen = ImageDataGenerator(rescale=1./255)
        self.seg_testY_datagen = ImageDataGenerator()
    def FromFolder(self,pathFolder,size,batch_size):
        train_generator,test_generator =self.train_datagen.flow_from_directory(
            pathFolder,  # this is the target directory
            target_size=(size[0], size[1]),  # all images will be resized to 150x150
            batch_size=batch_size,
            class_mode='categorical')
        return train_generator,test_generator
    def FromFolder(self,pathTrain,pathTest,size,batch_size):
        train_generator = self.train_datagen.flow_from_directory(
            pathTrain,  # this is the target directory
            target_size=(size[0], size[1]),  # all images will be resized to 150x150
            batch_size=batch_size,
            class_mode='categorical')
        validation_generator = self.test_datagen.flow_from_directory(
            pathTest,
            target_size=(size[0], size[1]),
            batch_size=batch_size,
            class_mode='categorical')
        return train_generator,validation_generator
    def SegmentationTrainFromImage(self,X_Train,Y_Train,seed,batch_size):
        
        x_train=self.seg_trainX_datagen.flow(X_Train,seed=seed,batch_size=batch_size)
        y_train=self.seg_trainY_datagen.flow(Y_Train,seed=seed,batch_size=batch_size)
        return zip(x_train,y_train)
    def SegmentationTestFromImage(self,X_Test,Y_Test,seed,batch_size):
        
        x_test=self.seg_testX_datagen.flow(X_Test,seed=seed,batch_size=batch_size)
        y_test=self.seg_testY_datagen.flow(Y_Test,seed=seed,batch_size=batch_size)
        return zip(x_test,y_test)
        

In [ ]:
 def tversky_loss(y_true, y_pred):
    alpha = 0.5
    beta  = 0.5
    ones = K.ones(K.shape(y_true))
    p0 = y_pred      # proba that voxels are class i
    p1 = ones-y_pred # proba that voxels are not class i
    g0 = y_true
    g1 = ones-y_true
    num = K.sum(p0*g0, (0,1,2,3))
    den = num + alpha*K.sum(p0*g1,(0,1,2,3)) + beta*K.sum(p1*g0,(0,1,2,3))
    T = K.sum(num/den) # when summing over classes, T has dynamic range [0 Ncl]
    Ncl = K.cast(K.shape(y_true)[-1], 'float32')
    return Ncl-T
def dice_coef(y_true, y_pred,smooth):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f*y_true_f) + K.sum(y_pred_f*y_pred_f) + smooth)
def dice_coef_loss(y_true, y_pred):
    return 1.-dice_coef(y_true, y_pred)

In [ ]:
VGG_Weights_path = "D:/Nam/vgg16_weights_tf_dim_ordering_tf_kernels.h5"

def VGGSegnet( n_classes ,  input_height=224, input_width=224 , vgg_level=5):

    img_input = Input(shape=(input_height,input_width,3))
    
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block1_conv1', data_format='channels_last' )(img_input)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block1_conv2', data_format='channels_last' )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block0_pool', data_format='channels_last' )(x)
    f0 = x

    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1', data_format='channels_last' )(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2', data_format='channels_last' )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool', data_format='channels_last' )(x)
    f1 = x
    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1', data_format='channels_last' )(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2', data_format='channels_last' )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool', data_format='channels_last' )(x)
    f2 = x

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1', data_format='channels_last' )(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2', data_format='channels_last' )(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3', data_format='channels_last' )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool', data_format='channels_last' )(x)
    f3 = x

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1', data_format='channels_last' )(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2', data_format='channels_last' )(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3', data_format='channels_last' )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool', data_format='channels_last' )(x)
    f4 = x

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1', data_format='channels_last' )(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2', data_format='channels_last' )(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3', data_format='channels_last' )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool', data_format='channels_last' )(x)
    f5 = x

    x = Flatten(name='flatten')(x)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense( 1000 , activation='softmax', name='predictions')(x)

    vgg  = Model(  img_input , x  )
    vgg.load_weights(VGG_Weights_path)

    levels = [f1 , f2 , f3 , f4 , f5 ]

    o = levels[ vgg_level ]

    o = ( ZeroPadding2D( (1,1) , data_format='channels_last' ))(o)
    o = ( Conv2D(512, (3, 3), padding='valid', data_format='channels_last'))(o)
    o = ( BatchNormalization())(o)
    
    o = ( UpSampling2D( (2,2), data_format='channels_last'))(o)
    o = ( ZeroPadding2D( (1,1), data_format='channels_last'))(o)
    o = ( Conv2D( 512, (3, 3), padding='valid', data_format='channels_last'))(o)
    o = ( BatchNormalization())(o)
    
    o = ( UpSampling2D( (2,2), data_format='channels_last'))(o)
    o = ( ZeroPadding2D( (1,1), data_format='channels_last'))(o)
    o = ( Conv2D( 256, (3, 3), padding='valid', data_format='channels_last'))(o)
    o = ( BatchNormalization())(o)

    o = ( UpSampling2D((2,2)  , data_format='channels_last' ) )(o)
    o = ( ZeroPadding2D((1,1) , data_format='channels_last' ))(o)
    o = ( Conv2D( 128 , (3, 3), padding='valid' , data_format='channels_last' ))(o)
    o = ( BatchNormalization())(o)

    o = ( UpSampling2D((2,2)  , data_format='channels_last' ))(o)
    o = ( ZeroPadding2D((1,1)  , data_format='channels_last' ))(o)
    o = ( Conv2D( 64 , (3, 3), padding='valid'  , data_format='channels_last' ))(o)
    o = ( BatchNormalization())(o)
    
    

    o =  Conv2D( n_classes , (3, 3) , padding='same', data_format='channels_last' )( o )
    #o_shape = Model(img_input , o ).output_shape
    #outputHeight = o_shape[2]
    #outputWidth = o_shape[3]

    #o = (Reshape((  -1  , outputHeight*outputWidth   )))(o)
    #o = (Permute((2, 1)))(o)
    o = (Activation('softmax'))(o)
    model = Model( img_input , o )
    #model.outputWidth = outputWidth
    #model.outputHeight = outputHeight

    return model

In [2]:
def VGGSegnet( n_classes ,  input_height=200, input_width=800 , vgg_level=5):

    img_input = Input(shape=(input_height,input_width,3))
    
    x = Resizing(224, 224, interpolation="bilinear", crop_to_aspect_ratio=False, **kwargs)(img_input)
    
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1', data_format='channels_last' )(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2', data_format='channels_last' )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool', data_format='channels_last' )(x)
    f1 = x
    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1', data_format='channels_last' )(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2', data_format='channels_last' )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool', data_format='channels_last' )(x)
    f2 = x

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1', data_format='channels_last' )(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2', data_format='channels_last' )(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3', data_format='channels_last' )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool', data_format='channels_last' )(x)
    f3 = x

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1', data_format='channels_last' )(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2', data_format='channels_last' )(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3', data_format='channels_last' )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool', data_format='channels_last' )(x)
    f4 = x

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1', data_format='channels_last' )(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2', data_format='channels_last' )(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3', data_format='channels_last' )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool', data_format='channels_last' )(x)
    f5 = x

    x = Flatten(name='flatten')(x)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense( 1000 , activation='softmax', name='predictions')(x)

    vgg  = Model(  img_input , x  )
    return vgg
model=VGGSegnet(5).summary()

NameError: name 'Resizing' is not defined

In [9]:
def get_small_unet(n_filters = 16, bn = True, dilation_rate = 1):
    '''Validation Image data generator
        Inputs: 
            n_filters - base convolution filters
            bn - flag to set batch normalization
            dilation_rate - convolution dilation rate
        Output: Unet keras Model
    '''
    #Define input batch shape
    batch_shape=(256,256,3)
    inputs = Input(batch_shape=(None, 256, 800, 3))
    print(inputs)
    
    conv1 = Conv2D(n_filters * 1, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(inputs)
    if bn:
        conv1 = BatchNormalization()(conv1)
        
    conv1 = Conv2D(n_filters * 1, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(conv1)
    if bn:
        conv1 = BatchNormalization()(conv1)
    
    pool1 = MaxPooling2D(pool_size=(2, 2), data_format='channels_last')(conv1)

    conv2 = Conv2D(n_filters * 2, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(pool1)
    if bn:
        conv2 = BatchNormalization()(conv2)
        
    conv2 = Conv2D(n_filters * 2, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(conv2)
    if bn:
        conv2 = BatchNormalization()(conv2)
    
    pool2 = MaxPooling2D(pool_size=(2, 2), data_format='channels_last')(conv2)

    conv3 = Conv2D(n_filters * 4, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(pool2)
    if bn:
        conv3 = BatchNormalization()(conv3)
        
    conv3 = Conv2D(n_filters * 4, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(conv3)
    if bn:
        conv3 = BatchNormalization()(conv3)
        
    pool3 = MaxPooling2D(pool_size=(2, 2), data_format='channels_last')(conv3)

    conv4 = Conv2D(n_filters * 8, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(pool3)
    if bn:
        conv4 = BatchNormalization()(conv4)
        
    conv4 = Conv2D(n_filters * 8, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(conv4)
    if bn:
        conv4 = BatchNormalization()(conv4)
        
    pool4 = MaxPooling2D(pool_size=(2, 2), data_format='channels_last')(conv4)

    conv5 = Conv2D(n_filters * 16, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(pool4)
    if bn:
        conv5 = BatchNormalization()(conv5)
        
    conv5 = Conv2D(n_filters * 16, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(conv5)
    if bn:
        conv5 = BatchNormalization()(conv5)
        
    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4], axis=3)
    
    conv6 = Conv2D(n_filters * 8, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(up6)
    if bn:
        conv6 = BatchNormalization()(conv6)
        
    conv6 = Conv2D(n_filters * 8, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(conv6)
    if bn:
        conv6 = BatchNormalization()(conv6)
        
    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv3], axis=3)
    
    conv7 = Conv2D(n_filters * 4, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(up7)
    if bn:
        conv7 = BatchNormalization()(conv7)
        
    conv7 = Conv2D(n_filters * 4, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(conv7)
    if bn:
        conv7 = BatchNormalization()(conv7)
        
    up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2], axis=3)
    
    conv8 = Conv2D(n_filters * 2, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(up8)
    if bn:
        conv8 = BatchNormalization()(conv8)
        
    conv8 = Conv2D(n_filters * 2, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(conv8)
    if bn:
        conv8 = BatchNormalization()(conv8)
        
    up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1], axis=3)
    
    conv9 = Conv2D(n_filters * 1, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(up9)
    if bn:
        conv9 = BatchNormalization()(conv9)
        
    conv9 = Conv2D(n_filters * 1, (3, 3), activation='relu', padding = 'same', dilation_rate = dilation_rate)(conv9)
    if bn:
        conv9 = BatchNormalization()(conv9)
        
    conv10 = Conv2D(5, (1, 1), activation='softmax', padding = 'same', dilation_rate = dilation_rate)(conv9)

    model = Model(inputs=inputs, outputs=conv10)

    
    return model

In [ ]:
VGG_Weights_path = "D:/Nam/vgg16_weights_tf_dim_ordering_tf_kernels.h5"

IMAGE_ORDERING = 'channels_last'


def FCN32( n_classes ,  input_height=224, input_width=224, vgg_level=3):

    assert input_height%32 == 0
    assert input_width%32 == 0

    # https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_th_dim_ordering_th_kernels.h5
    img_input = Input(shape=(input_height,input_width,3))

    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1', data_format=IMAGE_ORDERING )(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2', data_format=IMAGE_ORDERING )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool', data_format=IMAGE_ORDERING )(x)
    f1 = x
    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2', data_format=IMAGE_ORDERING )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool', data_format=IMAGE_ORDERING )(x)
    f2 = x

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3', data_format=IMAGE_ORDERING )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool', data_format=IMAGE_ORDERING )(x)
    f3 = x

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3', data_format=IMAGE_ORDERING )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool', data_format=IMAGE_ORDERING )(x)
    f4 = x

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3', data_format=IMAGE_ORDERING )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool', data_format=IMAGE_ORDERING )(x)
    f5 = x

    x = Flatten(name='flatten')(x)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense( 1000 , activation='softmax', name='predictions')(x)

    vgg  = Model(  img_input , x  )
    vgg.load_weights(VGG_Weights_path)

    o = f5

    o = ( Conv2D( 4096 , ( 7 , 7 ) , activation='relu' , padding='same', data_format=IMAGE_ORDERING))(o)
    o = Dropout(0.5)(o)
    o = ( Conv2D( 4096 , ( 1 , 1 ) , activation='relu' , padding='same', data_format=IMAGE_ORDERING))(o)
    o = Dropout(0.5)(o)

    o = ( Conv2D( n_classes ,  ( 1 , 1 ) ,kernel_initializer='he_normal' , data_format=IMAGE_ORDERING))(o)
    o = Conv2DTranspose( n_classes , kernel_size=(32,32) ,  strides=(32,32) , use_bias=False ,  data_format=IMAGE_ORDERING )(o)
    #o_shape = Model(img_input , o ).output_shape

    #outputHeight = o_shape[2]
    #outputWidth = o_shape[3]

    #print ("koko" , o_shape)

    #o = (Reshape(( -1  , outputHeight*outputWidth   )))(o)
    #o = (Permute((2, 1)))(o)
    o = (Activation('softmax'))(o)
    model = Model( img_input , o )
    #model.outputWidth = outputWidth
    #model.outputHeight = outputHeight

    return model

In [ ]:
model1 = VGGSegnet( 5, vgg_level=3)
model1.summary()

In [10]:
model2 = get_small_unet()
model2.summary()

Tensor("input_5:0", shape=(None, 256, 800, 3), dtype=float32)
Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 256, 800, 3) 0                                            
__________________________________________________________________________________________________
conv2d_57 (Conv2D)              (None, 256, 800, 16) 448         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_54 (BatchNo (None, 256, 800, 16) 64          conv2d_57[0][0]                  
__________________________________________________________________________________________________
conv2d_58 (Conv2D)              (None, 256, 800, 16) 2320        batch_normalization_54[0][0]     
_________________________

In [ ]:
model3 = FCN32( 5, vgg_level=3)
model3.summary()

In [ ]:
LoadData=LoadData();
pathFolder="D:/Nam/Train/Train";
xtrain,ytrain=LoadData.SegmentationFromFolder(pathFolder,"bmp","png",(224,224),"_color_mask")
x_train,x_test,y_train,y_test=train_test_split(xtrain,ytrain,test_size=0.2, random_state=20)
unique_pixels = np.vstack({tuple(r) for r in y_train[1].reshape(-1,3)})
unique_pixels


In [ ]:
ListColor=[[0,0,0],[255,255,255],[0,60,100],[119,11,32],[150,100,100],[70,70,70]]
ListID=[[0,0,0],[0,0,0],[0,60,100],[119,11,32],[150,100,100],[70,70,70]]
ListName=["BackGroud","BackGroud","Bridge","Fat","OK","Over"]
#Meger 2 backgound to 1 backgourd
Label=ImageLabel(ListColor,ListID,ListName)
y_id=Label.TranferListImageColorToListImageID(y_train)
y_train=None
y_test_id=Label.TranferListImageColorToListImageID(y_test)
y_test=None
unique_pixels = np.vstack({tuple(r) for r in y_id[1].reshape(-1,3)})
unique_pixels

In [ ]:
# One Hot Encoding
ListColor=[(0,0,0),(0,60,100),(119,11,32),(150,100,100),(70,70,70)]
ListID=[(0,0,0),(1,1,1),(2,2,2),(3,3,3),(4,4,4)]
ListName=["BackGroud","Bridge","Fat","OK","Over"]
#Meger 2 backgound to 1 backgourd
convertLabel=ImageLabel(ListColor,ListID,ListName)
convertLabel.id2code

In [ ]:
y_id[0].shape

In [ ]:
ytrain=convertLabel.ListImageToOneHot(y_id)
ytest=convertLabel.ListImageToOneHot(y_test_id)
ytrain.shape

In [ ]:
x_train.shape,ytrain.shape,x_test.shape,ytest.shape

In [ ]:
#Aug
Aug=PreProcessImageWithTensorflow()
DataGen_Train=Aug.SegmentationTrainFromImage(x_train,ytrain,1,16)
DataGen_Test=Aug.SegmentationTrainFromImage(x_test,ytest,1,16)

In [ ]:
tb = TensorBoard(log_dir='logs', write_graph=True)
mc = ModelCheckpoint(mode='max', filepath='D:/SegSoloder6.h5', monitor='accuracy', save_best_only='True', save_weights_only='True', verbose=1)
es = EarlyStopping(mode='min', monitor='val_loss', patience=50, verbose=1)
callbacks = [es,mc]


In [ ]:
batch_size = 16
steps_per_epoch = np.ceil(float(len(x_train) - round(0.2*len(x_train))) / float(batch_size))
steps_per_epoch

validation_steps = (float((round(0.2*len(x_train)))) / float(batch_size))
validation_steps


In [ ]:
num_epochs = 120
batch_size = 16
opt = Adam(lr=1e-3, decay=1e-3 / num_epochs)
model1.compile(optimizer=opt, loss="categorical_crossentropy",  metrics=['accuracy'])
with tf.device("/gpu:1"):
    result = model3.fit_generator(DataGen_Train, steps_per_epoch=steps_per_epoch ,
                validation_data = DataGen_Test, 
                validation_steps =validation_steps, epochs=num_epochs, callbacks=callbacks, verbose=1)
model3.save_weights('D:/SegSoloder6.h5', overwrite=True,shuffle=True)


In [ ]:
# Get actual number of epochs model was trained for
N = len(result.history['loss'])

#Plot the model evaluation history
plt.style.use("ggplot")
fig = plt.figure(figsize=(20,8))

fig.add_subplot(1,2,1)
plt.title("Training Loss")
plt.plot(np.arange(0, N), result.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), result.history["val_loss"], label="val_loss")
plt.ylim(0, 1)

fig.add_subplot(1,2,2)
plt.title("Training Accuracy")
plt.plot(np.arange(0, N), result.history["accuracy"], label="train_accuracy")
plt.plot(np.arange(0, N), result.history["val_accuracy"], label="val_accuracy")
plt.ylim(0, 1)

plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()

In [ ]:
model=load_model("D:/SegSoloder10.h5")

In [ ]:
origin=cv2.imread("D:/Project/ProjectCustomer/TaiDoDo/mask/Train/Image_0001659697511319.bmp",cv2.IMREAD_COLOR)
origin = cv2.cvtColor(origin, cv2.COLOR_BGR2RGB)
img= cv2.resize(origin, (224, 224),interpolation = cv2.INTER_NEAREST)

img=img/255
img=img.reshape(1,224,224,3)
mask=cv2.imread("D:/Project/ProjectCustomer/TaiDoDo/mask/Train/Image_0001659697511319_color_mask.png",cv2.IMREAD_COLOR)
mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)

pred_=model.predict(img)



In [ ]:
pred_.shape

In [ ]:
pred_=pred_.reshape(224,224,5)

In [ ]:
# One Hot Encoding
ListColor=[(0,0,0),(0,60,100),(119,11,32),(150,100,100),(70,70,70)]
ListID=[(0,0,0),(1,1,1),(2,2,2),(3,3,3),(4,4,4)]
ListName=["BackGroud","Bridge","Fat","OK","Over"]
#Meger 2 backgound to 1 backgourd
convertLabel=ImageLabel(ListColor,ListID,ListName)
convertLabel.id2code


In [ ]:
pred=convertLabel.OneHot_To_RGB(pred_)
pred= cv2.resize(pred, (480, 500),interpolation = cv2.INTER_NEAREST)
pred = cv2.cvtColor(pred, cv2.COLOR_BGR2RGB)

In [ ]:
fig = plt.figure(figsize=(10,8))

fig.add_subplot(1,3,1)
plt.title("Origin")
plt.imshow(origin)

fig.add_subplot(1,3,2)
plt.title("Mask")
plt.imshow(mask)

fig.add_subplot(1,3,3)
plt.title("Predict")
plt.imshow(mask)
plt.show()

In [ ]:
img[0,100,100]

In [ ]:
pred_[100,100,1]